In [ ]:
import sys
import os
import numpy as np
import open3d as o3d

SCRIPT_DIR = os.getcwd()
REPO_DIR = os.path.dirname(SCRIPT_DIR)
sys.path.append(REPO_DIR)

from utils import FileManager, fit_circle_to_pcd
from frame import Frame
from visualizer import ImageVisualizer, generic_o3d_vis

RUN_DIR_PATH = os.path.join(REPO_DIR, "sample_data", "run_01")

if __name__ == "__main__":
    fm = FileManager(RUN_DIR_PATH)
    imgvis = ImageVisualizer()

    idx = 0

    pair = fm.get_pair(idx)
    frame = Frame(
        frame_idx=idx,
        depth_path=pair.depth_path,
        color_path=pair.color_path,
        raw_timestamp=pair.raw_timestamp,
        recording_time_str=pair.recording_time_str
    )

In [ ]:
# Visualize raw depth and color images
imgvis.depth(frame.raw_depth_img)
imgvis.color(frame.color_img)


In [ ]:
# Visualize raw point cloud data
raw_pcd = frame.all_pixels.create_raw_pcd()
generic_o3d_vis(raw_pcd)

In [ ]:
# Visualize the systemic depth error
pixels = frame.all_pixels
raw_depth = pixels.create_raw_depth_img()

bin_size = 25
min_depth = np.min(raw_depth[raw_depth > 0])
max_depth = np.max(raw_depth)
bin_edges = np.arange(min_depth, max_depth + bin_size, bin_size)

for i in range(0, len(bin_edges) - 1):
    bin_min = bin_edges[i]
    bin_max = bin_edges[i + 1]
    bin_pixels = pixels.get_pixels_in_range_raw_depth(bin_min, bin_max)
    bin_img = bin_pixels.create_raw_depth_img()
    imgvis.depth(bin_img)

In [ ]:
# Visualize the raw depth image (foreground pixels)
fg_img = frame.all_pixels.get_foreground_pixels().create_raw_depth_img()
imgvis.depth(fg_img)

In [ ]:
# Visualize the raw point cloud data (foreground pixels)
fg_pcd = frame.all_pixels.get_foreground_pixels().create_raw_pcd()
generic_o3d_vis(fg_pcd)

In [ ]:
# Visualize the raw depth image (background pixels)
bg_img = frame.all_pixels.get_background_pixels().create_raw_depth_img()
imgvis.depth(bg_img)

In [ ]:
# Visualize the raw point cloud data (foreground pixels)
bg_pcd = frame.all_pixels.get_background_pixels().create_raw_pcd()
generic_o3d_vis(bg_pcd)

In [ ]:
# Visualize the corrected depth image
depth_corrected_depth_img = frame.all_pixels.create_depth_corrected_depth_img()
imgvis.depth_corrected(depth_corrected_depth_img)

In [ ]:
# Visualize the depth image with corrected and filtered pixels
filtered_pixels = frame._filter_pixels(0, 0)

depth_corrected_depth_img = filtered_pixels.create_depth_corrected_depth_img()
imgvis.depth_corrected(depth_corrected_depth_img)

In [ ]:
# Visualize the filtered point cloud with depth correction applied
filtered_pcd = frame._filter_pixels(100, 100).create_depth_corrected_pcd()
generic_o3d_vis(filtered_pcd)

In [ ]:
# Visualize the filtered depth-corrected point cloud and the fitted circles for each bin
pixels = frame._filter_pixels(199, 149)
depth_corrected_img = pixels.create_depth_corrected_depth_img()

bin_size = 5
min_depth = np.min(depth_corrected_img[depth_corrected_img > 0])
max_depth = np.max(depth_corrected_img)
bin_edges = np.arange(min_depth, max_depth + bin_size, bin_size)

geometries = []
centers = []

for i in range(0, len(bin_edges) - 1):
    bin_min = bin_edges[i]
    bin_max = bin_edges[i + 1]
    bin_pixels = pixels.get_pixels_in_range_corrected_depth(bin_min, bin_max)
    bin_depth_img = bin_pixels.create_depth_corrected_depth_img()
    bin_pcd = bin_pixels.create_depth_corrected_pcd()

    if len(bin_pcd.points) == 0:
        continue

    geometries.append(bin_pcd)

    center, radius, mean_depth = fit_circle_to_pcd(bin_pcd)

    if center is not None and radius is not None and mean_depth is not None:
        centers.append([center[0], center[1], mean_depth])

        num_circle_points = 36
        angles = np.linspace(0, 2*np.pi, num_circle_points, endpoint=False)
        x = center[0] + radius * np.cos(angles)
        y = center[1] + radius * np.sin(angles)
        z = np.full_like(x, mean_depth)
        circle_points = np.vstack((x, y, z)).T

        lines = []
        for j in range(num_circle_points):
            lines.append([j, (j+1)%num_circle_points])

        circle_lineset = o3d.geometry.LineSet(
            points=o3d.utility.Vector3dVector(circle_points),
            lines=o3d.utility.Vector2iVector(lines)
        )
        circle_lineset.paint_uniform_color([1, 0, 0])
        geometries.append(circle_lineset)

if len(centers) >= 2:
    centers_points = o3d.utility.Vector3dVector(centers)
    lines = [[i, i+1] for i in range(len(centers)-1)]

    center_line = o3d.geometry.LineSet(
        points=centers_points,
        lines=o3d.utility.Vector2iVector(lines)
    )
    center_line.paint_uniform_color([0, 0, 1])
    geometries.append(center_line)

generic_o3d_vis(geometries)
